# March Madness 2025

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import os
from sklearn.model_selection import train_test_split
import random
from data import Data, STATS_COLUMNS
import model
import elo

torch.manual_seed(20250222)
random.seed(20250222)
np.random.seed(20250222)

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


## Hypothesis
Each team can be modeled by x hidden features. In each game, these hidden features interact in a nonlinear fashion to determine the outcome of the game

## Preparing the data
Load the data

In [2]:
dataset = Data(batch_size=100)

dataset.games.describe()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,NumOT,WFGM,WFGA,WFGM3,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
count,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,...,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000
mean,2015.575896,70.371624,2101.677721,74.190697,2099.323680,61.205668,0.061738,26.180557,57.072441,6.919323,...,19.266279,11.437051,16.819468,10.801152,21.937435,11.218625,15.191485,6.454031,2.849489,18.836014
std,6.073611,36.089915,986.770174,11.403172,989.989621,11.373177,0.287218,4.809783,7.820486,3.168482,...,6.327385,5.238806,6.985267,4.417465,4.706791,3.764948,5.029754,2.984269,2.036512,4.586616
min,2003.000000,0.000000,1101.000000,30.000000,1101.000000,11.000000,0.000000,9.000000,26.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,3.000000
25%,2011.000000,38.000000,1261.000000,66.000000,1254.000000,54.000000,0.000000,23.000000,52.000000,5.000000,...,15.000000,8.000000,12.000000,8.000000,19.000000,9.000000,12.000000,4.000000,1.000000,16.000000
50%,2016.000000,74.000000,1413.000000,74.000000,1407.000000,61.000000,0.000000,26.000000,57.000000,7.000000,...,19.000000,11.000000,16.000000,10.000000,22.000000,11.000000,15.000000,6.000000,3.000000,19.000000
75%,2021.000000,101.000000,3245.000000,81.000000,3246.000000,69.000000,0.000000,29.000000,62.000000,9.000000,...,23.000000,15.000000,21.000000,14.000000,25.000000,14.000000,18.000000,8.000000,4.000000,22.000000
max,2025.000000,132.000000,3480.000000,149.000000,3480.000000,144.000000,6.000000,58.000000,113.000000,30.000000,...,80.000000,48.000000,65.000000,38.000000,53.000000,34.000000,49.000000,26.000000,21.000000,47.000000


The x's will be the indexes of two team IDs and program IDs, the y's will include and indicator of who won at the game stats

## Predicting Stats
The first model we build will predict game statistics. There is more information encapsulated in the box scores than simply wins and losses so we will use this to build up embeddings that are predictive about how the games will play out

In [3]:
stats_train_loader, stats_validation_loader = dataset.train_test_data(output_stats=True, cache="stats", train_size=0.99)

Loading cached data


In [4]:
stats_model = model.StatsModel(program_embedding=512, team_embedding=512,
                               num_programs=len(dataset.programs), num_teams=len(dataset.teams),
                               model_sizes=(512, 256), dropout=0.5).to(device)

In [5]:
model.train(stats_train_loader, stats_validation_loader, stats_model, name="stats_model", learning_rate=0.001, use_cache=True)

Loading from cache


## The Model
Now we use the embeddings developed in the first model to build a model to predict which team will win, which is the thing we are, in the end trying to do.

In [6]:
result_train_loader, result_validation_loader = dataset.train_test_data(output_stats=False, cache="result", train_size=0.99, seed=20250320)

Loading cached data


In [7]:
result_model = model.Model(matchup=stats_model.matchup, model_sizes=(384, 32), dropout=0.5).to(device)
result_model.matchup.freeze()

## Training the model

Train the model

In [8]:
model.train(result_train_loader, result_validation_loader, result_model, name="result_model", learning_rate=0.0001)

Epoch   0: Train Loss=0.24881859, Test Loss=0.24883938
Epoch   1: Train Loss=0.16938005, Test Loss=0.17200699
Epoch   2: Train Loss=0.16415690, Test Loss=0.16759954
Epoch   3: Train Loss=0.16248913, Test Loss=0.16621562
Epoch   4: Train Loss=0.16414274, Test Loss=0.16760223
Epoch   5: Train Loss=0.16271739, Test Loss=0.16623589
Epoch   6: Train Loss=0.16380225, Test Loss=0.16746641
Epoch   7: Train Loss=0.16083402, Test Loss=0.16491942
Epoch   8: Train Loss=0.16227809, Test Loss=0.16604713
Epoch   9: Train Loss=0.16250004, Test Loss=0.16647970
Epoch  10: Train Loss=0.16230731, Test Loss=0.16640287
Epoch  11: Train Loss=0.16291618, Test Loss=0.16710496
Epoch  12: Train Loss=0.16210578, Test Loss=0.16664009
Best Loss: 0.164919


In [9]:
model.print_results(result_validation_loader, result_model, label="Validation Results")

Validation Results: Accuracy=75.42, Loss=0.164919


With this model we can predict the output of about three quarters of regular season games.

## Load the tourney data to test with

In [10]:
tourney_dataset = dataset.gen_dataset(dataset.tourney)
tourney_loader = DataLoader(tourney_dataset, batch_size=500, shuffle=True)

In [11]:
model.print_results(tourney_loader, result_model, label="Tournament Results")

Tournament Results: Accuracy=73.77, Loss=0.169402


When it comes to tournament results we get a little worse. The lower result is likely due to teams having increased pairity.

In [12]:
stage1_loader = dataset.tourney_data(after=2021)
model.print_results(stage1_loader, result_model, label=f"Stage 1")

Stage 1: Accuracy=71.94, Loss=0.176030


Train with early torney data

In [13]:
tourney_df = dataset.tourney[dataset.tourney.Season < 2021]

tourney_train_df, tourney_validation_df = train_test_split(tourney_df, train_size=0.8)
tourney_train_data = dataset.gen_dataset(tourney_train_df)
tourney_validation_data = dataset.gen_dataset(tourney_validation_df)

tourney_train_loader = DataLoader(tourney_train_data, batch_size=500)
tourney_validation_loader = DataLoader(tourney_validation_data, batch_size=500)

In [14]:
result_model.matchup.freeze()

In [15]:
model.train(tourney_train_loader, tourney_validation_loader, result_model, name="tuned_model", learning_rate=0.0001)

Epoch   0: Train Loss=0.16850019, Test Loss=0.18120284
Epoch   1: Train Loss=0.16619061, Test Loss=0.17887312
Epoch   2: Train Loss=0.16478221, Test Loss=0.17726855
Epoch   3: Train Loss=0.16461945, Test Loss=0.17701522
Epoch   4: Train Loss=0.16493289, Test Loss=0.17723300
Epoch   5: Train Loss=0.16508848, Test Loss=0.17735451
Epoch   6: Train Loss=0.16502304, Test Loss=0.17735909
Epoch   7: Train Loss=0.16477234, Test Loss=0.17719668
Epoch   8: Train Loss=0.16446371, Test Loss=0.17704411
Best Loss: 0.177015


In [16]:
model.print_results(stage1_loader, result_model, label=f"Stage 1")

Stage 1: Accuracy=71.66, Loss=0.175859


### Performance by year


In [17]:
for season in dataset.tourney.Season.unique():
    loader = dataset.tourney_data(year=season)
    model.print_results(loader, result_model, label=f"{season} Tournament")

2003 Tournament: Accuracy=75.78, Loss=0.166490
2004 Tournament: Accuracy=70.31, Loss=0.183649
2005 Tournament: Accuracy=75.00, Loss=0.174487
2006 Tournament: Accuracy=66.41, Loss=0.202839
2007 Tournament: Accuracy=73.44, Loss=0.177222
2008 Tournament: Accuracy=78.91, Loss=0.167437
2009 Tournament: Accuracy=76.56, Loss=0.153106
2010 Tournament: Accuracy=70.87, Loss=0.173079
2011 Tournament: Accuracy=74.62, Loss=0.168615
2012 Tournament: Accuracy=76.15, Loss=0.158760
2013 Tournament: Accuracy=70.77, Loss=0.176898
2014 Tournament: Accuracy=74.62, Loss=0.162298
2015 Tournament: Accuracy=79.23, Loss=0.142090
2016 Tournament: Accuracy=75.00, Loss=0.169759
2017 Tournament: Accuracy=79.23, Loss=0.153743
2018 Tournament: Accuracy=68.46, Loss=0.188438
2019 Tournament: Accuracy=75.77, Loss=0.150102
2021 Tournament: Accuracy=71.32, Loss=0.179505
2022 Tournament: Accuracy=74.25, Loss=0.170110
2023 Tournament: Accuracy=69.78, Loss=0.187507
2024 Tournament: Accuracy=71.27, Loss=0.166448


Breaking out by league

In [18]:
for season in dataset.tourney.Season.unique():
    for league in dataset.tourney[dataset.tourney.Season == season].League.unique():
        loader = dataset.tourney_data(year=season, league=league)
        model.print_results(loader, result_model, label=f"{season} {league} Tournament")

2003 M Tournament: Accuracy=75.78, Loss=0.166490
2004 M Tournament: Accuracy=70.31, Loss=0.183649
2005 M Tournament: Accuracy=75.00, Loss=0.174487
2006 M Tournament: Accuracy=66.41, Loss=0.202839
2007 M Tournament: Accuracy=73.44, Loss=0.177222
2008 M Tournament: Accuracy=78.91, Loss=0.167437
2009 M Tournament: Accuracy=76.56, Loss=0.153106
2010 M Tournament: Accuracy=67.97, Loss=0.202302
2010 W Tournament: Accuracy=73.81, Loss=0.143392
2011 M Tournament: Accuracy=67.91, Loss=0.215889
2011 W Tournament: Accuracy=81.75, Loss=0.118339
2012 M Tournament: Accuracy=70.15, Loss=0.188095
2012 W Tournament: Accuracy=82.54, Loss=0.127563
2013 M Tournament: Accuracy=65.67, Loss=0.203679
2013 W Tournament: Accuracy=76.19, Loss=0.148416
2014 M Tournament: Accuracy=67.16, Loss=0.200977
2014 W Tournament: Accuracy=82.54, Loss=0.121163
2015 M Tournament: Accuracy=76.12, Loss=0.160593
2015 W Tournament: Accuracy=82.54, Loss=0.122412
2016 M Tournament: Accuracy=73.88, Loss=0.185158
2016 W Tournament: A

## Inspect the model
First what are the sizes of the smallest input and output weights

In [19]:
print(f"Program embedding min: {result_model.matchup.program_embedding.state_dict()['weight'].abs().max(axis=0).values.min().item():>8f}")
print(f"Team embedding min: {result_model.matchup.team_embedding.state_dict()['weight'].abs().max(axis=0).values.min().item():>8f}")
print(f"FC min: {result_model.fc3.state_dict()['weight'].abs().max(axis=0).values.min().item():>8f}")

Program embedding min: 2.373395
Team embedding min: 3.467458
FC min: 0.014745


Calculate the average gradient for each input feature

In [20]:
result_model.matchup.unfreeze()

In [21]:
program_weights, team_weights, stats_weights = model.feature_eval(result_model, tourney_loader)

In [22]:
program_weights.abs().sum().item(), team_weights.abs().sum().item()

(0.0002996377006638795, 0.0002748886472545564)

In [23]:
print(f"Year:\t{stats_weights[0]:>4f}")
print(f"Game:\t{stats_weights[1]:>4f}")
print(f"League:\t{stats_weights[2]:>4f}")

Year:	-0.000078
Game:	-0.000197
League:	-0.008605


## Save the model

In [24]:
torch.save(result_model.state_dict(), 'model.pth')

## Moderated model

Moderate a model by pushing it towards 0.5

In [25]:
moderated = model.ModeratedModel(result_model, 0.75)
model.print_results(stage1_loader, moderated, label="Moderated Model")

Moderated Model: Accuracy=71.66, Loss=0.180755


In [26]:
for season in dataset.tourney.Season.unique():
    loader = dataset.tourney_data(season)
    model.print_results(loader, moderated, label=f"{season} Tournament")

2003 Tournament: Accuracy=75.78, Loss=0.175213
2004 Tournament: Accuracy=70.31, Loss=0.188372
2005 Tournament: Accuracy=75.00, Loss=0.180085
2006 Tournament: Accuracy=66.41, Loss=0.204156
2007 Tournament: Accuracy=73.44, Loss=0.184794
2008 Tournament: Accuracy=78.91, Loss=0.177008
2009 Tournament: Accuracy=76.56, Loss=0.165862
2010 Tournament: Accuracy=70.87, Loss=0.178527
2011 Tournament: Accuracy=74.62, Loss=0.174106
2012 Tournament: Accuracy=76.15, Loss=0.168308
2013 Tournament: Accuracy=70.77, Loss=0.181667
2014 Tournament: Accuracy=74.62, Loss=0.170002
2015 Tournament: Accuracy=79.23, Loss=0.154623
2016 Tournament: Accuracy=75.00, Loss=0.175353
2017 Tournament: Accuracy=79.23, Loss=0.162929
2018 Tournament: Accuracy=68.46, Loss=0.188190
2019 Tournament: Accuracy=75.77, Loss=0.160406
2021 Tournament: Accuracy=71.32, Loss=0.182446
2022 Tournament: Accuracy=74.25, Loss=0.177457
2023 Tournament: Accuracy=69.78, Loss=0.190179
2024 Tournament: Accuracy=71.27, Loss=0.173001


It underperforms even in years where there were upsets.

## Dig into 2023 results

In [27]:
loader = dataset.tourney_data(2023)

x, y = loader.dataset.tensors

preds = result_model(x.to(device))

In [28]:
t_2023 = pd.DataFrame({'winner_name': [dataset.all_teams.loc[dataset.programs.loc[i].TeamID].TeamName for i in x[:,0].tolist()],
                       'loser_name': [dataset.all_teams.loc[dataset.programs.loc[i].TeamID].TeamName for i in x[:,2].tolist()],
                       'winner': [dataset.programs.loc[i].TeamID for i in x[:,0].tolist()],
                       'loser': [dataset.programs.loc[i].TeamID for i in x[:,2].tolist()],
                       'actual': y.reshape(-1),
                       'predicted': preds.cpu().detach().numpy().reshape(-1)}).iloc[:67]

In [29]:
t_2023[t_2023.predicted < 0.5].sort_values('predicted')

,winner_name,loser_name,winner,loser,actual,predicted
23,F Dickinson,Purdue,1192,1345,1.0,0.015271
15,Princeton,Arizona,1343,1112,1.0,0.105450
8,Furman,Virginia,1202,1438,1.0,0.196113
53,FL Atlantic,Tennessee,1194,1397,1.0,0.267572
39,Princeton,Missouri,1343,1281,1.0,0.286336
58,San Diego St,Alabama,1361,1104,1.0,0.294405
57,Miami FL,Houston,1274,1222,1.0,0.321572
50,Michigan St,Marquette,1277,1266,1.0,0.367600
14,Penn St,Texas A&M,1336,1401,1.0,0.372930
63,San Diego St,Creighton,1361,1166,1.0,0.394807


The biggest thing in this season were the huge upsets in the first round. Purdue was a number one seed and lost which I only gave a 1% chance to happen. Arizona and Virginia were number 2 seeds and lost which I gave 13% and 17% chances of happening respectively.

In [30]:
t_2023['Upset'] = [dataset.upset(2023, winner, loser) for (winner, loser) in zip(t_2023['winner'], t_2023['loser'])]

In [31]:
t_2023[t_2023.Upset].predicted.mean()

np.float64(0.3694488912563835)

On average the upsets had a 37% chance of happening

In [32]:
t_2023[t_2023.Upset & (t_2023.predicted >= 0.5)].sort_values('predicted', ascending=False)

,winner_name,loser_name,winner,loser,actual,predicted,Upset
45,Creighton,Baylor,1166,1124,1.0,0.570718,True
60,Connecticut,Gonzaga,1163,1211,1.0,0.564477,True
49,Miami FL,Indiana,1274,1231,1.0,0.501356,True


I correctly predicted 4 upsets, though all were closely ranked

In [33]:
t_2023[~t_2023.Upset & (t_2023.predicted < 0.5)].sort_values('predicted')

,winner_name,loser_name,winner,loser,actual,predicted,Upset
63,San Diego St,Creighton,1361,1166,1.0,0.394807,False
12,Missouri,Utah St,1281,1429,1.0,0.437063,False
0,Pittsburgh,Mississippi St,1338,1280,1.0,0.456306,False
2,Arizona St,Nevada,1113,1305,1.0,0.456842,False
59,Texas,Xavier,1400,1462,1.0,0.474238,False
5,Arkansas,Illinois,1116,1228,1.0,0.492024,False
48,Kansas St,Kentucky,1243,1246,1.0,0.494278,False


I also incorrectly predicted 5 upsets

Looking at all the tourneys

In [34]:
x, y = tourney_loader.dataset.tensors
preds = result_model(x.to(device))
tourney_df = pd.DataFrame({'season': x[:,4].tolist(),
                           'winner_name': [dataset.all_teams.loc[dataset.programs.loc[i].TeamID].TeamName for i in x[:,0].tolist()],
                           'loser_name': [dataset.all_teams.loc[dataset.programs.loc[i].TeamID].TeamName for i in x[:,2].tolist()],
                           'winner': [dataset.programs.loc[i].TeamID for i in x[:,0].tolist()],
                           'loser': [dataset.programs.loc[i].TeamID for i in x[:,2].tolist()],
                           'actual': y.reshape([-1]),
                           'predicted': np.array(preds.tolist()).reshape([-1])})
tourney_df = tourney_df[tourney_df.actual == 1.0]
tourney_df['Upset'] = [dataset.upset(season, winner, loser) for (winner, loser, season)
                       in zip(tourney_df['winner'], tourney_df['loser'], tourney_df['season'])]

In [35]:
len(tourney_df[tourney_df.Upset & (tourney_df.predicted >= 0.5) & (tourney_df.season > 2020)].sort_values('predicted', ascending=False))

27

In [36]:
len(tourney_df[~tourney_df.Upset & (tourney_df.predicted < 0.5) & (tourney_df.season > 2020)].sort_values('predicted'))

43

Overall I predicted 25 upsets correctly, and 36 incorrectly

## Predicting by seeds
What if I predict just using the seeds?

In [37]:
odds = dataset.odds_by_seed_diff(before=2021)

In [38]:
dataset.tourney_df(after=2021).SeedDiff.map(lambda x: odds[x]**2).mean()

np.float64(0.18527460145235355)

This results in a test Brier score of about 0.185.

## Using Elo
I can also compute Elo rankings

In [39]:
elo_model = elo.EloModel(dataset, k=50)
model.print_results(stage1_loader, elo_model, label="Elo Model")

Elo Model: Accuracy=73.45, Loss=0.178886


It is similar to our model

## Hybrid Model
Building a model using the neural net and seeds

In [40]:
seed_model = model.SeedModel(dataset)
model.print_results(stage1_loader, seed_model, label="Seed Model")
model.print_results(stage1_loader, elo_model, label="Elo Model")
model.print_results(stage1_loader, result_model, label="Base NN")

Seed Model: Accuracy=72.03, Loss=0.184022
Elo Model: Accuracy=73.45, Loss=0.178886
Base NN: Accuracy=71.66, Loss=0.175859


In [41]:
hybrid = model.HybridModel([result_model, seed_model, elo_model], [0.7, 0.2, 0.1])

model.print_results(stage1_loader, hybrid, label="Hybrid Model")

Hybrid Model: Accuracy=73.54, Loss=0.174659


They hybrid model outperforms all individual models

In [42]:
for season in range(2021, 2025):
    for league in ['M', 'W']:
        loader = dataset.tourney_data(year=season, league=league)
        model.print_results(loader, hybrid, label=f"{season} {league} Tournament")

2021 M Tournament: Accuracy=68.94, Loss=0.209730
2021 W Tournament: Accuracy=77.78, Loss=0.149420
2022 M Tournament: Accuracy=71.64, Loss=0.201304
2022 W Tournament: Accuracy=78.36, Loss=0.141393
2023 M Tournament: Accuracy=69.40, Loss=0.203721
2023 W Tournament: Accuracy=76.12, Loss=0.166271
2024 M Tournament: Accuracy=64.93, Loss=0.193715
2024 W Tournament: Accuracy=81.34, Loss=0.130731


## Generating the submission file
### Phase 2

Write the results

In [43]:
model.gen_submission(hybrid, dataset, fname="hybrid.csv")

In [44]:
model.gen_submission(result_model, dataset, fname="base.csv")

## Generate a bracket

With the hybrid model

In [45]:
season, league = 2025, 'M'

In [46]:
bracket = model.gen_bracket(dataset, season, league, hybrid).join(dataset.all_teams, on='Winner')[['Winner', 'TeamName', 'P']]
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(bracket)

      Winner        TeamName         P
Slot                                  
R1W1    1181            Duke  0.988571
R1W2    1104         Alabama  0.902399
R1W3    1458       Wisconsin  0.909682
R1W4    1112         Arizona  0.860633
R1W5    1332          Oregon  0.761523
R1W6    1140             BYU  0.560719
R1W7    1388    St Mary's CA  0.663156
R1W8    1280  Mississippi St  0.511135
R1X1    1222         Houston  0.981250
R1X2    1397       Tennessee  0.939480
R1X3    1246        Kentucky  0.870813
R1X4    1345          Purdue  0.814709
R1X5    1155         Clemson  0.811335
R1X6    1228        Illinois  0.656811
R1X7    1417            UCLA  0.577876
R1X8    1211         Gonzaga  0.621978
R1Y1    1120          Auburn  0.991566
R1Y2    1277     Michigan St  0.937645
R1Y3    1235         Iowa St  0.865923
R1Y4    1401       Texas A&M  0.766161
R1Y5    1276        Michigan  0.658669
R1Y6    1314  North Carolina  0.539235
R1Y7    1266       Marquette  0.585068
R1Y8    1257      Louisvi

In [47]:
model.tiebreaker(bracket, stats_model, dataset, season, league)

140.15078814263353

And with the base model

In [48]:
base_bracket = model.gen_bracket(dataset, 2025, 'M', result_model).join(dataset.all_teams, on='Winner')[['Winner', 'TeamName', 'P']]
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(base_bracket)

      Winner        TeamName         P
Slot                                  
R1W1    1181            Duke  0.989775
R1W2    1104         Alabama  0.882510
R1W3    1458       Wisconsin  0.902102
R1W4    1112         Arizona  0.865436
R1W5    1332          Oregon  0.758375
R1W6    1140             BYU  0.520019
R1W7    1388    St Mary's CA  0.656668
R1W8    1280  Mississippi St  0.509907
R1X1    1222         Houston  0.978175
R1X2    1397       Tennessee  0.931978
R1X3    1246        Kentucky  0.848863
R1X4    1345          Purdue  0.795338
R1X5    1155         Clemson  0.819162
R1X6    1228        Illinois  0.670615
R1X7    1417            UCLA  0.560319
R1X8    1211         Gonzaga  0.634015
R1Y1    1120          Auburn  0.992976
R1Y2    1277     Michigan St  0.930358
R1Y3    1235         Iowa St  0.837799
R1Y4    1401       Texas A&M  0.737306
R1Y5    1276        Michigan  0.615780
R1Y6    1314  North Carolina  0.594330
R1Y7    1266       Marquette  0.565851
R1Y8    1257      Louisvi

In [49]:
model.tiebreaker(base_bracket, stats_model, dataset, season, league)

140.15078814263353

Women's bracket

In [50]:
season, league = 2025, 'W'
bracket = model.gen_bracket(dataset, season, league, hybrid).join(dataset.all_teams, on='Winner')[['Winner', 'TeamName', 'P']]
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(bracket)

      Winner        TeamName         P
Slot                                  
R1W1    3376  South Carolina  0.987182
R1W2    3181            Duke  0.963826
R1W3    3314  North Carolina  0.894958
R1W4    3268        Maryland  0.859462
R1W5    3104         Alabama  0.743075
R1W6    3452   West Virginia  0.758468
R1W7    3435      Vanderbilt  0.687302
R1W8    3428            Utah  0.534326
R1X1    3400           Texas  0.992336
R1X2    3395             TCU  0.977019
R1X3    3323      Notre Dame  0.966947
R1X4    3326         Ohio St  0.876933
R1X5    3397       Tennessee  0.845943
R1X6    3276        Michigan  0.625051
R1X7    3257      Louisville  0.551182
R1X8    3166       Creighton  0.586805
R1Y1    3417            UCLA  0.987875
R1Y2    3301        NC State  0.940300
R1Y3    3261             LSU  0.938488
R1Y4    3124          Baylor  0.863746
R1Y5    3279     Mississippi  0.747616
R1Y6    3199      Florida St  0.758305
R1Y7    3277     Michigan St  0.707538
R1Y8    3210    Georgia T

In [51]:
bracket = model.gen_bracket(dataset, season, league, result_model).join(dataset.all_teams, on='Winner')[['Winner', 'TeamName', 'P']]
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(bracket)

      Winner        TeamName         P
Slot                                  
R1W1    3376  South Carolina  0.986467
R1W2    3181            Duke  0.965509
R1W3    3314  North Carolina  0.888143
R1W4    3268        Maryland  0.855386
R1W5    3104         Alabama  0.747574
R1W6    3452   West Virginia  0.808438
R1W7    3435      Vanderbilt  0.708061
R1W8    3428            Utah  0.544414
R1X1    3400           Texas  0.993559
R1X2    3395             TCU  0.983954
R1X3    3323      Notre Dame  0.975735
R1X4    3326         Ohio St  0.876339
R1X5    3397       Tennessee  0.878410
R1X6    3276        Michigan  0.625778
R1X7    3257      Louisville  0.508384
R1X8    3166       Creighton  0.604719
R1Y1    3417            UCLA  0.987758
R1Y2    3301        NC State  0.931551
R1Y3    3261             LSU  0.938817
R1Y4    3124          Baylor  0.866156
R1Y5    3279     Mississippi  0.748539
R1Y6    3199      Florida St  0.787772
R1Y7    3277     Michigan St  0.728643
R1Y8    3210    Georgia T

## |Actual Results

In [108]:
actual_games = np.array([dataset.matchup(winner, loser, 2025, league) for winner, loser, league in
                         [# Mens
                          # Round 1
                          [1120,1106,'M'],[1166,1257,'M'],[1276,1471,'M'],[1401,1463,'M'],[1235,1252,'M'],#South
                          [1181,1291,'M'],[1140,1433,'M'],[1458,1285,'M'],[1124,1277,'M'],[1104,1352,'M'],#East
                          [1161,1272,'M'],[1179,1281,'M'],[1403,1423,'M'],[1116,1242,'M'],[1385,1303,'M'],#West
                          [1222,1188,'M'],[1211,1208,'M'],[1270,1155,'M'],[1345,1219,'M'],[1417,1429,'M'],[1397,1459,'M']#Midwest
                         ]])
actual_results = np.ones((len(actual_games),1))
actual_ds = TensorDataset(torch.from_numpy(actual_games), torch.from_numpy(actual_results))
actual_loader = DataLoader(actual_ds)
model.print_results(actual_loader, hybrid, label="Actual Hybrid Model")
model.print_results(actual_loader, result_model, label="Actual Base Model")
model.print_results(actual_loader, moderated, label="Actual Moderated Model")
model.print_results(actual_loader, elo_model, label="Actual Elo Model")
model.print_results(actual_loader, seed_model, label="Actual Seeds Model")


Actual Hybrid Model: Accuracy=71.43, Loss=0.161531
Actual Base Model: Accuracy=71.43, Loss=0.168684
Actual Moderated Model: Accuracy=71.43, Loss=0.171515
Actual Elo Model: Accuracy=85.71, Loss=0.133248
Actual Seeds Model: Accuracy=71.43, Loss=0.161353


### Men's ELO Rankings

In [103]:
elo_rankings = [(team, elo_model.ratings[dataset.teamMapping[(team, 2025)]],) 
                for team in dataset.seeds.loc[2025].loc['M'].index]
for team, ranking in sorted(elo_rankings):
    print(f"{dataset.all_teams.loc[team].TeamName} ({team}): {ranking:.2f}")

Akron (1103): 1238.11
Alabama (1104): 1612.31
Alabama St (1106): 788.55
American Univ (1110): 933.42
Arizona (1112): 1525.12
Arkansas (1116): 1405.07
Auburn (1120): 1659.61
Baylor (1124): 1422.12
Bryant (1136): 1049.31
BYU (1140): 1565.44
Clemson (1155): 1515.53
Colorado St (1161): 1486.80
Connecticut (1163): 1557.59
Creighton (1166): 1534.52
Drake (1179): 1447.69
Duke (1181): 1670.73
SIUE (1188): 887.49
Florida (1196): 1699.45
Georgia (1208): 1385.18
Gonzaga (1211): 1517.88
Grand Canyon (1213): 1258.72
High Point (1219): 1166.02
Houston (1222): 1770.81
Illinois (1228): 1501.04
Iowa St (1235): 1534.19
Kansas (1242): 1453.77
Kentucky (1246): 1503.35
Liberty (1251): 1249.57
Lipscomb (1252): 1094.69
Louisville (1257): 1481.30
Marquette (1266): 1483.97
Maryland (1268): 1521.87
McNeese St (1270): 1155.86
Memphis (1272): 1480.30
Michigan (1276): 1538.41
Michigan St (1277): 1620.11
Mississippi (1279): 1445.07
Mississippi St (1280): 1382.01
Missouri (1281): 1365.28
Montana (1285): 1126.47
Mt S

## Women's Elo Rankings

In [97]:
elo_rankings = [(elo_model.ratings[dataset.teamMapping[(team, 2025)]],team) 
                for team in dataset.seeds.loc[2025].loc['W'].index]
for ranking, team in sorted(elo_rankings, reverse=True):
    print(f"{dataset.all_teams.loc[team].TeamName} ({team}): {ranking:.2f}")

South Carolina (3376): 2051.64
Texas (3400): 1908.29
UCLA (3417): 1895.33
Connecticut (3163): 1881.40
USC (3425): 1874.32
Notre Dame (3323): 1788.93
Duke (3181): 1774.16
NC State (3301): 1763.91
LSU (3261): 1748.59
Ohio St (3326): 1712.88
TCU (3395): 1687.75
Oklahoma (3328): 1680.38
Creighton (3166): 1666.87
Iowa (3234): 1666.54
North Carolina (3314): 1655.81
Baylor (3124): 1650.63
Maryland (3268): 1644.01
Louisville (3257): 1613.82
Michigan (3276): 1607.29
Mississippi (3279): 1604.18
Florida St (3199): 1593.95
West Virginia (3452): 1579.69
Tennessee (3397): 1574.51
Indiana (3231): 1568.17
California (3143): 1558.56
S Dakota St (3355): 1553.67
Iowa St (3235): 1539.89
Michigan St (3277): 1538.06
Kansas St (3243): 1537.08
Alabama (3104): 1536.62
Kentucky (3246): 1532.42
Oklahoma St (3329): 1532.40
Utah (3428): 1521.25
Nebraska (3304): 1515.99
Vanderbilt (3435): 1505.10
Illinois (3228): 1490.10
Columbia (3162): 1471.12
FGCU (3195): 1463.76
Princeton (3343): 1457.45
Richmond (3350): 1448.9